In [1]:
import pandas as pd
import sys
sys.path.append('..')
from scripts import utils
from scripts import constants
from langchain_community.llms import Ollama
import os
import openai
from openai import OpenAI

In [2]:
test_df = pd.read_csv(constants.TEST_SET_PATH)
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [3]:
myllm = Ollama(model="mistral")
#myllm = Ollama(model="llama3")

#### Plain without COT

In [4]:
llm_results_df = pd.DataFrame(columns=['y_actual', 'y_pred'])
llm_results_df['y_actual'] = test_df[:1000]['label']
llm_results_df = llm_results_df.replace({"y_actual": constants.CLASS_DICT})
llm_results_df['y_pred'] = llm_results_df.apply(lambda row: utils.get_diagnosis(row.name, myllm, constants.plainNOCOT_prompt), axis=1)
llm_results_df.head()

,y_actual,y_pred
0,hemolytic anemia,hemolytic anemia
1,vitamin b12/folate deficiency anemia,vitamin b12/folate deficiency anemia
2,iron deficiency anemia,anemia of chronic disease
3,no anemia,no anemia
4,inconclusive diagnosis,anemia of chronic disease


In [5]:
acc_spec, f1_spec, roc_auc_spec = utils.test(llm_results_df.y_actual, llm_results_df.y_pred)
acc_spec, f1_spec, roc_auc_spec

(36.1, 28.07030958724373, 62.892614674821054)

In [6]:
llm_results_df.to_csv('final_plainNOCOT_llama_results.csv', index=False, header=False)
#llm_results_df.to_csv('final_plainNOCOT_mistral_results.csv', index=False, header=False)

#### Plain with COT

In [4]:
# Create the pathway dataframe to store all responses
pathway_dataframe = pd.DataFrame(columns=['patient_idx', 'llm_response'])

# Create results_df with the extracted diagnosis
results_df = pd.DataFrame(columns=['y_actual', 'y_pred'])

results_df['y_actual'] = test_df[:1000]['label']
# results_df['y_actual'] = test_df[100:200]['label'].values

results_df = results_df.replace({"y_actual": constants.CLASS_DICT})

In [5]:
for idx in range(1000):
    # Get diagnosis
    full_response, diagnosis = utils.get_diagnosis_cot(idx, myllm, constants.plainCOT_prompt)
    
    # Store full response in pathway dataframe
    new_row = pd.DataFrame({'patient_idx': [idx], 'llm_response': [full_response]})
    pathway_dataframe = pd.concat([pathway_dataframe, new_row], ignore_index=True)
    
    # Store extracted diagnosis in results_df['y_pred']
    results_df.at[idx, 'y_pred'] = diagnosis


Processing patient:  0
Processing patient:  1
Processing patient:  2
Processing patient:  3
Processing patient:  4
Processing patient:  5
Processing patient:  6
Processing patient:  7
Processing patient:  8
Processing patient:  9
Processing patient:  10
Processing patient:  11
Processing patient:  12
Processing patient:  13
Processing patient:  14
Processing patient:  15
Processing patient:  16
Processing patient:  17
Processing patient:  18
Processing patient:  19
Processing patient:  20
Processing patient:  21
Processing patient:  22
Processing patient:  23
Processing patient:  24
Processing patient:  25
Processing patient:  26
Processing patient:  27
Processing patient:  28
Processing patient:  29
Processing patient:  30
Processing patient:  31
Processing patient:  32
Processing patient:  33
Processing patient:  34
Processing patient:  35
Processing patient:  36
Processing patient:  37
Processing patient:  38
Processing patient:  39
Processing patient:  40
Processing patient:  41
Pr

In [6]:
pathway_dataframe.head()

,patient_idx,llm_response
0,0,Based on the provided laboratory results and ...
1,1,The diagnosis for this patient is Vitamin B12...
2,2,Based on the provided laboratory results and ...
3,3,Based on the provided laboratory results and ...
4,4,Based on the provided laboratory results and ...


In [7]:
results_df.head()

,y_actual,y_pred
0,hemolytic anemia,hemolytic anemia
1,vitamin b12/folate deficiency anemia,vitamin b12/folate deficiency anemia
2,iron deficiency anemia,inconclusive diagnosis
3,no anemia,hemolytic anemia
4,inconclusive diagnosis,inconclusive diagnosis


In [8]:
acc_spec, f1_spec, roc_auc_spec = utils.test(results_df.y_actual, results_df.y_pred)
acc_spec, f1_spec, roc_auc_spec

(29.299999999999997, 26.272425012639122, 59.56372223094228)

In [9]:
# Combine results_df and pathway_dataframe
final_df = results_df.copy()
final_df['llm_response'] = pathway_dataframe['llm_response']

# Save the final DataFrame to a CSV file
#final_df.to_csv("plainCOT_llama_full_result.csv", index=False)
final_df.to_csv("plainCOT_mistral_full_result.csv")


# Select only the y_actual and y_pred columns
llm_results = results_df[["y_actual", "y_pred"]]
#llm_results.to_csv('final_plainCOT_llama_results.csv', index=False, header=False)
llm_results.to_csv('final_plainCOT_mistral_results.csv', index=False, header=False)


In [10]:
results_df

,y_actual,y_pred
0,hemolytic anemia,hemolytic anemia
1,vitamin b12/folate deficiency anemia,vitamin b12/folate deficiency anemia
2,iron deficiency anemia,inconclusive diagnosis
3,no anemia,hemolytic anemia
4,inconclusive diagnosis,inconclusive diagnosis
...,...,...
995,anemia of chronic disease,hemolytic anemia
996,anemia of chronic disease,hemolytic anemia
997,aplastic anemia,hemolytic anemia
998,hemolytic anemia,hemolytic anemia


#### ChatGPT Plain No COT 

In [11]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", 'sk-proj-vFWubWoE4Ckt34AMvlW3T3BlbkFJj2Cfxl3uqOSsn9Jyb01v'))

In [12]:
query = f"""

Question: {constants.plainNOCOT_prompt}
So now consider this particular patient and give me the pathway for the final diagnosis:"
"""

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about Anemia.'},
        {'role': 'user', 'content': query},
    ],
    model="gpt-4-turbo",
    temperature=0,
)
results_df = utils.get_results_chatgpt(250, "gpt-4-turbo", query, True, filename = 'final_plainNOCOT_chatgpt_results.csv')

Processing patient:  0
Processing patient:  1
Processing patient:  2
Processing patient:  3
Processing patient:  4
Processing patient:  5
Processing patient:  6
Processing patient:  7
Processing patient:  8
Processing patient:  9
Processing patient:  10
Processing patient:  11
Processing patient:  12
Processing patient:  13
Processing patient:  14
Processing patient:  15
Processing patient:  16
Processing patient:  17
Processing patient:  18
Processing patient:  19
Processing patient:  20
Processing patient:  21
Processing patient:  22
Processing patient:  23
Processing patient:  24
Processing patient:  25
Processing patient:  26
Processing patient:  27
Processing patient:  28
Processing patient:  29
Processing patient:  30
Processing patient:  31
Processing patient:  32
Processing patient:  33
Processing patient:  34
Processing patient:  35
Processing patient:  36
Processing patient:  37
Processing patient:  38
Processing patient:  39
Processing patient:  40
Processing patient:  41
Pr

In [15]:
results_df.head()

,y_actual,y_pred
0,hemolytic anemia,hemolytic anemia
1,vitamin b12/folate deficiency anemia,vitamin b12/folate deficiency anemia
2,iron deficiency anemia,iron deficiency anemia
3,no anemia,no anemia
4,inconclusive diagnosis,vitamin b12/folate deficiency anemia


In [16]:
acc_a, f1_a, roc_auc_a = utils.test(results_df.y_actual, results_df.y_pred)
acc_a, f1_a, roc_auc_a

(82.0, 74.90702225015187, 87.49049257946034)

#### ChatGPT Plain COT

In [17]:
query = f"""

Question: {constants.plainCOT_prompt}
So now consider this particular patient and give me the pathway for the final diagnosis:"
"""

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about Anemia.'},
        {'role': 'user', 'content': query},
    ],
    model="gpt-4-turbo",
    temperature=0,
)
results_df = utils.get_results_chatgpt_cot(250, "gpt-4-turbo", query, True, filename = 'final_plainCOT_chatgpt_results.csv')

Processing patient:  0
Processing patient:  1
Processing patient:  2
Processing patient:  3
Processing patient:  4
Processing patient:  5
Processing patient:  6
Processing patient:  7
Processing patient:  8
Processing patient:  9
Processing patient:  10
Processing patient:  11
Processing patient:  12
Processing patient:  13
Processing patient:  14
Processing patient:  15
Processing patient:  16
Processing patient:  17
Processing patient:  18
Processing patient:  19
Processing patient:  20
Processing patient:  21
Processing patient:  22
Processing patient:  23
Processing patient:  24
Processing patient:  25
Processing patient:  26
Processing patient:  27
Processing patient:  28
Processing patient:  29
Processing patient:  30
Processing patient:  31
Processing patient:  32
Processing patient:  33
Processing patient:  34
Processing patient:  35
Processing patient:  36
Processing patient:  37
Processing patient:  38
Processing patient:  39
Processing patient:  40
Processing patient:  41
Pr

In [18]:
results_df.head()

,y_actual,y_pred
0,hemolytic anemia,hemolytic anemia
1,vitamin b12/folate deficiency anemia,vitamin b12/folate deficiency anemia
2,iron deficiency anemia,iron deficiency anemia
3,no anemia,no anemia
4,inconclusive diagnosis,inconclusive diagnosis


In [19]:
acc_a, f1_a, roc_auc_a = utils.test(results_df.y_actual, results_df.y_pred)
acc_a, f1_a, roc_auc_a

(92.80000000000001, 91.40610701470197, 94.90461504480032)